In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix

%matplotlib inline

In [2]:
data='/Users/alejandro/AnacondaProjects/Actuary Data/train_set.csv'
df = pd.read_csv(data, nrows=100000)

# dummy code claims
df['claim'] = df.Claim_Amount.apply(lambda x: 0 if x == 0 else 1)
df['Model_Year'] = df['Model_Year'] - np.min(df['Model_Year'])
df['Model_Year'] = df['Model_Year'].astype('float64')
df.head()

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount,claim
0,1,1,3,2005,24.0,K,K.78,K.78.2,D,C,...,0.261040,0.907793,-0.077998,M,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
1,2,2,2,2005,22.0,Q,Q.22,Q.22.3,B,C,...,0.432987,-0.726459,0.204785,O,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
2,3,3,1,2005,17.0,AR,AR.41,AR.41.1,B,?,...,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
3,4,3,1,2006,17.0,AR,AR.41,AR.41.1,B,?,...,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
4,5,3,2,2005,20.0,D,D.20,D.20.0,J,C,...,2.112691,1.534462,2.347260,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0


In [3]:
X = df[['Cat1', 'Cat2', 'NVCat', 'Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4', 'Vehicle', 'Model_Year', 'Blind_Model']]
y = df[['claim', 'Claim_Amount']].values

In [4]:
n_samples = y.shape[0]
n_classes = 2
Y = [int(y[i, 0]) for i in range(len(y[:, 0]))]
w1, w2 = n_samples / (n_classes * np.bincount(Y))

In [5]:
def pipeline(X):
    df_cont = pd.DataFrame()
    df_cat = pd.DataFrame()
    for x in X:
        if X[x].dtypes == 'float64':
            df_cont['{}'.format(x)] = X[x]
        else:
            df_cat['{}'.format(x)] = X[x]
    
    dummy = OneHotEncoder()
    dummyC = LabelEncoder()
    
    df_cat2 = np.zeros((df_cat.shape[0], 1))
    for x in df_cat:
        y = dummyC.fit_transform(df_cat[x].reshape(-1, 1))
        y = dummy.fit_transform(y.reshape(-1, 1)).toarray()
        y = pd.DataFrame(y[:, 1:])
        df_cat2 = np.hstack((df_cat2, y))
    df_cat = pd.DataFrame(df_cat2)
    
    df = pd.concat([df_cat, pd.DataFrame(df_cont)], axis=1)
    
    return df

In [6]:
X = pipeline(X)

/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
/anaconda3/envs/cv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
input_shape = X.shape[1]

In [8]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import backend as K

Using TensorFlow backend.


In [56]:
learning_rate = 0.001
batch_size = 256
epochs = 20

In [57]:
myAdam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0)

model = Sequential()

model.add(Dense(512, input_shape=(input_shape,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.8))


model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=myAdam,
              metrics=['accuracy'])

In [58]:
model.fit(X, y[:, 0], 
          epochs=epochs,
          batch_size=batch_size, 
          validation_split=0.15, 
          class_weight={0: w1, 1: w2}, 
          shuffle=True)

Train on 85000 samples, validate on 15000 samples
Epoch 1/20
85000/85000 [==============================] - 74s 872us/step - loss: 1.3492 - acc: 0.4758 - val_loss: 0.6292 - val_acc: 0.9753
Epoch 2/20
85000/85000 [==============================] - 72s 847us/step - loss: 0.9468 - acc: 0.4456 - val_loss: 0.6690 - val_acc: 0.9945
Epoch 3/20
85000/85000 [==============================] - 73s 856us/step - loss: 0.8575 - acc: 0.4435 - val_loss: 0.6383 - val_acc: 0.9876
Epoch 4/20
85000/85000 [==============================] - 72s 848us/step - loss: 0.8033 - acc: 0.4310 - val_loss: 0.6505 - val_acc: 0.9945
Epoch 5/20
85000/85000 [==============================] - 73s 855us/step - loss: 0.8205 - acc: 0.4657 - val_loss: 0.6314 - val_acc: 0.9771
Epoch 6/20
85000/85000 [==============================] - 71s 840us/step - loss: 0.7784 - acc: 0.4528 - val_loss: 0.6324 - val_acc: 0.9813
Epoch 7/20
85000/85000 [==============================] - 71s 839us/step - loss: 0.7878 - acc: 0.4093 - val_loss: 0.

In [59]:
y_probs = model.predict(X)

In [72]:
pred = pd.DataFrame(y_probs)
pred = pred.loc[:,1].apply(lambda x: 1 if x > 0.25 else 0)

In [73]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y[:, 0], pred))

,0,1
0,7704,91677
1,23,596
